# UDS emulator 

## Preliminaries

In [1]:
from my_imports import *

The current branch is: Badly_Implemented_Temporal_Bundling


In [2]:
yaml_path = 'config_file.yaml'
yaml_data = utils.load_yaml(yaml_path)

training_simulations_path       = yaml_data['training_simulations_path']
validation_simulations_path     = yaml_data['validation_simulations_path']

inp_path                        = yaml_data['inp_path']

training_windows_path           = yaml_data['training_windows_path']
use_saved_training_windows      = yaml_data['use_saved_training_windows']

validation_windows_path         = yaml_data['validation_windows_path']
use_saved_validation_windows    = yaml_data['use_saved_validation_windows']

num_events_training             = yaml_data['num_events_training']
num_events_validation           = yaml_data['num_events_validation']

steps_behind                    = yaml_data['steps_behind']
steps_ahead                     = yaml_data['steps_ahead']
epochs                          = yaml_data['epochs']



## Create simulation objects from data

In [3]:
list_of_training_simulations    = os.listdir(training_simulations_path)
list_of_validation_simulations  = os.listdir(validation_simulations_path)

random.shuffle(list_of_training_simulations)

print('Number of training simulations: ', len(list_of_training_simulations))
print('Number of validation simulations: ', len(list_of_validation_simulations))

Number of training simulations:  161
Number of validation simulations:  29


In [4]:
training_simulations = utils.extract_simulations_from_folders(training_simulations_path, inp_path, num_events_training)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [5]:
if use_saved_training_windows:
    training_windows = utils.load_windows(training_windows_path)
else:
    training_windows = utils.get_all_windows_from_list_simulations(training_simulations, steps_ahead, steps_behind)

In [6]:
# utils.save_pickle(training_windows, 'saved_objects/training_windows_debugging.pk')

In [7]:
validation_simulations = utils.extract_simulations_from_folders(validation_simulations_path, inp_path, num_events_validation)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [8]:
if use_saved_validation_windows:
    validation_windows = utils.load_windows(validation_windows_path)
else:
    validation_windows = utils.get_all_windows_from_list_simulations(validation_simulations, steps_ahead = 60, steps_behind = steps_behind)

In [9]:
print('Number of training windows: ',   len(training_windows))
print('Number of validation windows: ', len(validation_windows))

Number of training windows:  1676
Number of validation windows:  37


## Normalizer

In [10]:
normalizer = Normalizer(training_windows)

In [11]:
normalized_training_windows = normalizer.get_list_normalized_training_windows()
training_loader = DataLoader(normalized_training_windows, batch_size=32, shuffle = True)

In [12]:
normalized_validation_windows = [normalizer.normalize_window(val_win) for val_win in validation_windows]
validation_loader = DataLoader(normalized_validation_windows, batch_size=32)

## Training GNN

In [15]:
gnn_model = GNNModel(steps_ahead=1, steps_behind=6)
# gnn_model.load_state_dict(torch.load('saved_objects/model_trained_4steps.pt'))

optimizer = optim.Adam(gnn_model.parameters(), lr = 0.01)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
loss_fn   = nn.MSELoss()

In [16]:
history = train(
    model       = gnn_model,
    optimizer   = optimizer,
    scheduler   = scheduler,
    loss_fn     = loss_fn,
    train_dl    = training_loader,
    val_dl      = validation_loader,
    epochs      = epochs,
    device      = 'cpu',
    report_freq = 2
    )

train() called:model=GNNModel, opt=Adam(lr=0.010000), epochs=20,device=cpu

Best model so far at epoch:  1
Epoch   1/ 20, train loss:  4.42, val loss:  2.09
Epoch   2/ 20, train loss:  0.46, val loss:  2.30
Best model so far at epoch:  3
Epoch   4/ 20, train loss:  0.29, val loss:  1.64
Epoch   6/ 20, train loss:  0.19, val loss:  1.58
Epoch   8/ 20, train loss:  0.18, val loss:  1.00
Epoch  10/ 20, train loss:  0.17, val loss:  1.02
Best model so far at epoch:  11
Epoch  12/ 20, train loss:  0.15, val loss:  1.18
Epoch  14/ 20, train loss:  0.14, val loss:  3.26
Epoch  16/ 20, train loss:  0.14, val loss:  1.64
Epoch  18/ 20, train loss:  0.13, val loss:  0.87
Best model so far at epoch:  20
Epoch  20/ 20, train loss:  0.13, val loss:  0.60
Best model found at epoch:  20
This best model has this validation loss:  0.5976412695285084

Time total:     647.73 sec
Time per epoch: 32.39 sec


In [ ]:
# torch.save(gnn_model.state_dict(), 'saved_objects/trained_model_{}_epochs_{}_steps.pt'.format(epochs, steps_ahead))

In [17]:
vis.plot_loss(history)

## Using the GNN

In [18]:
val_event =  validation_simulations[-1]
print(val_event)

runoff_pd           = val_event.runoff_raw_data

sim_in_window       = val_event.get_simulation_in_one_window(steps_behind = 1)

norm_sim_in_window  = normalizer.normalize_window(sim_in_window)

swmm_heads_pd       = normalizer.get_unnormalized_heads_pd(sim_in_window['y']) 
predicted_heads_pd  = normalizer.get_unnormalized_heads_pd(gnn_model(norm_sim_in_window))

SWMMSimulation(real_90)


In [ ]:
importlib.reload(vis)

In [33]:
node =  'j_90369'
vis.plot_heads_timeseries(swmm_heads_pd, predicted_heads_pd, runoff_pd, node)

## Visualization of results

In [ ]:
importlib.reload(vis)

In [ ]:
error = predicted_heads_pd - swmm_heads_pd

In [ ]:
error= error.transpose().to_numpy()

In [ ]:
error = abs(error)
fig = vis.plot_nodal_variable(error, norm_sim_in_window, 'error in head (m) ', colorscale = 'OrRd')


In [ ]:
norm_sim_in_window.name_nodes[0]

In [ ]:
# fig.write_html('error.html')

In [ ]:
fig = vis.plot_nodal_variable(swmm_heads_pd.transpose().to_numpy(), norm_sim_in_window, 'Target Head ', colorscale = 'PuBu', ref_marker_size = 2.5)

# Scratchpad